<a href="https://colab.research.google.com/github/Gongsoo/Python3/blob/main/titanic2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
cd "/content/drive/MyDrive/"

/content/drive/MyDrive


In [ ]:
!unzip -qq "/content/drive/MyDrive/타이타닉.zip"

In [ ]:
!mkdir titanic_pytorch


In [ ]:
import shutil
shutil.move('/content/drive/MyDrive/train.csv','/content/drive/MyDrive/titanic_pytorch/train.csv')
shutil.move('/content/drive/MyDrive/test.csv','/content/drive/MyDrive/titanic_pytorch/test.csv')
shutil.move('/content/drive/MyDrive/submission.csv','/content/drive/MyDrive/titanic_pytorch/submission.csv')

'/content/drive/MyDrive/titanic_pytorch/submission.csv'

In [ ]:
shutil.move('/content/drive/MyDrive/타이타닉.zip','/content/drive/MyDrive/titanic_pytorch/titanic_data.zip')

'/content/drive/MyDrive/titanic_pytorch/titanic_data.zip'

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/train.csv')
test = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/titanic_pytorch/submission.csv')

In [4]:
train = train.drop('Ticket',axis = 1)
train = train.drop('Cabin',axis = 1)
test = test.drop('Ticket',axis = 1)
test = test.drop('Cabin',axis = 1)

In [5]:
train = train.fillna({'Embarked':'S'})
test = test.fillna({'Embarked':'S','Fare':20})

In [6]:
em_map = {'S':1,'Q':2,'C':3}

for t in [train] :
  t['Embarked'] = t['Embarked'].map(em_map)
for t in [test] :
  t['Embarked'] = t['Embarked'].map(em_map)

In [7]:
s_map = {'male':1, 'female':2}

for t in [train] :
  t['Sex'] = t['Sex'].map(s_map)
for t in [test] :
  t['Sex'] = t['Sex'].map(s_map)

In [8]:
age_map = {'Mr.':20,'Miss.':15,'Mrs.':20,'Ms.':10,'Mme.':30,'Mlle.':15,'Dona.':30,'Countess.':30,'Sir.':25,'Lady.':20,'Jonkheer.':20,'Don.':20,'Col.':35,'Major.':30,'Capt.':23,'Dr.':30,'Master.':25,'Rev.':35}

for t in [train] :
  for i in range(len(t['Age'])) :
    if np.isnan(t['Age'][i]) :
      for j in age_map :
        if j in t['Name'][i] :
          t['Age'][i] = age_map[j]
for t in [test] :
  for i in range(len(t['Age'])) :
    if np.isnan(t['Age'][i]) :
      for j in age_map :
        if j in t['Name'][i] :
          t['Age'][i] = age_map[j]

<ipython-input-8-69fb9fd690b4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['Age'][i] = age_map[j]
<ipython-input-8-69fb9fd690b4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['Age'][i] = age_map[j]


In [9]:
train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)

In [10]:
train = train.drop(['PassengerId'], axis=1)
test = test.drop(['PassengerId'], axis=1)

In [11]:
x = train.drop(['Survived'],axis=1)
y = train['Survived']

In [12]:
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [13]:
scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

scaler.fit(test)
test_scaled = scaler.transform(test)

In [44]:
print(type(y))

<class 'pandas.core.series.Series'>


In [14]:
train_x = torch.Tensor(x_scaled).float()
train_y = torch.Tensor(y).float()
test_x = torch.Tensor(test.values).float()

train_dataset = TensorDataset(train_x,train_y)
train_data_loader = DataLoader(train_dataset,batch_size = 64, shuffle = True,drop_last = True)


In [15]:
class LiModel(nn.Module) :
  def __init__(self):
    super(LiModel,self).__init__()
    self.layer1 = nn.Sequential(
        nn.Linear(7,512),
        nn.ReLU(),
        nn.Dropout(0.2)
    )
    self.layer2 = nn.Sequential(
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Dropout()
    )
    self.layer3 = nn.Sequential(
        nn.Linear(512,2)
    )
    
  
  def forward(self, x) :
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    return x
model = LiModel()
model

LiModel(
  (layer1): Sequential(
    (0): Linear(in_features=7, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (layer2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=512, out_features=2, bias=True)
  )
)

In [16]:

epochs = 500
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
train_loss_min = np.inf
for epoch in range(epochs) :
  train_loss = 0
  correct = 0
  for x,y in train_data_loader :
    
    model.train()
    optimizer.zero_grad()
    pred = model(x)
    loss = criterion(pred,y.long())

    
    loss.backward()
    optimizer.step()
    output = torch.argmax(pred, dim=1)
    correct += output.float() == y
    train_loss += loss.item() * len(x)
  train_loss = train_loss / len(train_x)
  if (epoch + 1) % 20 == 0:
      print('Epoch {}/{}, Prediction : {}/{}, Cost : {}'.format(epoch+1, epochs, sum(correct), len(train_x), train_loss))
  if train_loss < train_loss_min:
      print('=*=*=*= Loss decreased ({:10f} ===> {:10f}). Saving the model! =*=*=*='.format(train_loss_min, train_loss))
      torch.save(model.state_dict(), 'model.pt')
      train_loss_min = train_loss

=*=*=*= Loss decreased (       inf ===>   0.715587). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.715587 ===>   0.418927). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.418927 ===>   0.408882). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.408882 ===>   0.406069). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.406069 ===>   0.386640). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.386640 ===>   0.379643). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.379643 ===>   0.379469). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.379469 ===>   0.377122). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.377122 ===>   0.364861). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.364861 ===>   0.354287). Saving the model! =*=*=*=
Epoch 20/500, Prediction : 690/891, Cost : 0.37985830381930746
=*=*=*= Loss decreased (  0.354287 ===>   0.349403). Saving the model! =*=*=*=
=*=*=*= Loss decreased (  0.349403 ===>   0.345565). Saving the mode

In [24]:
with torch.no_grad():
  test_y_pred = model(test_x)
test_y_pred = torch.argmax(test_y_pred, dim = 1)

In [25]:
print(test_y_pred)

tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [26]:
sub['Survived'] = torch.Tensor.detach(test_y_pred)
sub.to_csv('베이스 라인 .csv', index = False)